<a href="https://colab.research.google.com/github/Git2723122/y4-dissertation/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 8.7 MB/s eta 0:00:00


In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Input
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape((x_train.shape[0], 28, 28 ,1))
x_test = x_test.reshape((x_test.shape[0], 28, 28 ,1))

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
def build_model(hp):
  model = Sequential([
      Conv2D(filters=hp.Int('filters1', 16, 64, step=16), kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
      MaxPooling2D((2,2)),
      Dropout(rate=hp.Float('rate1',0.1, 0.5, step=0.1)),

      Conv2D(filters=hp.Int('filters2', 32, 128, step=32), kernel_size=(3,3), activation='relu',input_shape=(28,28,1)),
      MaxPooling2D((2,2)),
      Dropout(rate=hp.Float('rate2',0.1,0.5,step=0.1)),

      Flatten(),
      Dense(units=hp.Int('units',64,256, step=64), activation='relu'),
      Dropout(rate=hp.Float('rate3',0.1,0.5,step=0.1)),

      Dense(10,activation='softmax')
  ])
  model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuning',
    project_name='tuning'
)

tuner.search(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

tuned_hps = tuner.get_best_hyperparameters(num_trials=3)[0]
print(f"Best Hyperparameters: {tuned_hps.values}")
tuned_model = tuner.hypermodel.build(tuned_hps)
tuned_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

In [7]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(28,28,1)))
  model.add(Conv2D(filters=hp.Int('filters_base', 16, 64, step=16), kernel_size=(3,3), activation='relu'))
  model.add(MaxPooling2D((2,2)))
  model.add(Dropout(rate=hp.Float('rate_base', 0.1, 0.5, step=0.1)))

  for i in range(hp.Int('num_layers', 1, 3)):
    model.add(Conv2D(filters=hp.Int(f'filters_{i}', 32, 128, step=32), kernel_size=(3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(rate=hp.Float(f'rate_{i}', 0.1, 0.5, step=0.1)))

  model.add(Flatten())
  model.add(Dense(units=hp.Int('units', 64, 256, step=64), activation='relu'))
  model.add(Dropout(rate=hp.Float('rate_dense', 0.1, 0.5, step=0.1)))

  model.add(Dense(10, activation='softmax'))

  model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

  return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='tuning',
    project_name='tuning'
)

tuner.search(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

tuned_hps = tuner.get_best_hyperparameters(num_trials=3)[0]
print(f"Best Hyperparameters: {tuned_hps.values}")
tuned_model = tuner.hypermodel.build(tuned_hps)
tuned_model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 01s]

Best val_accuracy So Far: 0.9610000252723694
Total elapsed time: 00h 03m 19s
Best Hyperparameters: {'filters_base': 16, 'rate_base': 0.30000000000000004, 'num_layers': 2, 'filters_0': 64, 'rate_0': 0.4, 'units': 192, 'rate_dense': 0.30000000000000004, 'learning_rate': 0.001, 'filters_1': 64, 'rate_1': 0.5, 'filters_2': 32}
Epoch 1/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 48s 24ms/step - accuracy: 0.6232 - loss: 1.0878 - val_accuracy: 0.9630 - val_loss: 0.1296
Epoch 2/2
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 24ms/step - accuracy: 0.9064 - loss: 0.3147 - val_accuracy: 0.9745 - val_loss: 0.0884
